# COMP4651 Project — Kaggle competition https://www.kaggle.com/c/ga-customer-revenue-prediction/overview
### - Data collected from https://www.kaggle.com/qnkhuat/make-data-ready/code
### - Those data are already cleaned!!!
### - Please read the code within the website to understand what is cleaned and processed!!!

In [6]:
import numpy as np
import pandas as pd
import sys
import os
import os.path as osp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

sns.set()

%matplotlib inline

In [7]:
train_data = None
test_data = None

for i in range(9):
    if i == 0:
        train_data = pd.read_pickle(osp.join("..", "input", "make-data-ready", "train_"+str(i)+".pkl"))
        continue
    else:
        temp = pd.read_pickle(osp.join("..", "input", "make-data-ready", "train_"+str(i)+".pkl"))
        train_data = pd.concat([train_data, temp], ignore_index=True)

for j in range(3):
    if j == 0:
        test_data = pd.read_pickle(osp.join("..", "input", "make-data-ready", "test_"+str(j)+".pkl"))
        continue
    else:
        temp = pd.read_pickle(osp.join("..", "input", "make-data-ready", "test_"+str(j)+".pkl"))
        test_data = pd.concat([test_data, temp], ignore_index=True)
    


In [8]:
print(type(train_data))
print(type(test_data))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [9]:
print(train_data.columns)

Index(['channelGrouping', 'date', 'fullVisitorId', 'visitId', 'visitNumber',
       'visitStartTime', 'device_browser', 'device_deviceCategory',
       'device_isMobile', 'device_operatingSystem', 'geoNetwork_city',
       'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_metro',
       'geoNetwork_networkDomain', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_sessionQualityDim',
       'totals_timeOnSite', 'totals_totalTransactionRevenue',
       'totals_transactionRevenue', 'totals_transactions', 'totals_visits',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
       'trafficSource_isTrueDirect', 'trafficSource_keyword',
       '

In [ ]:
sorted_revenue = train_data['totals_transactionRevenue'].sort_values(ascending=False)
top_10_revenue = sorted_revenue.head(10)
least_10_revenue = sorted_revenue.tail(10)

print(top_10_revenue, '\n')
print(least_10_revenue, '\n')
print(sorted_revenue.describe(), '\n')

# train_data.plot.scatter(x='visitId', y='totals_transactionRevenue')
plt.scatter(x=range(len(sorted_revenue)), y=sorted_revenue.sort_values(ascending=True))

plt.title("Total transaction Revenue of each customers")
plt.xlabel("visitor count")
plt.ylabel("Total Transaction Revenue")

plt.show()

In [ ]:
Top10_browser_used = train_data.groupby('device_browser')['visitId'].nunique().sort_values(ascending=False).head(10)
Top10_browser_used.plot.bar()

plt.title("Top 10 Browser Used")
plt.xlabel("device browser")
plt.ylabel("number of users")

plt.show()

Top10_nonzero_revenue_browser_used = train_data[train_data.totals_transactionRevenue>0].groupby('device_browser')['visitId'].nunique().sort_values(ascending=False).head(10)

Top10_nonzero_revenue_browser_used.plot.bar()

plt.title("Top 10 Non Zero Revenue Browser Used")
plt.xlabel("device browser")
plt.ylabel("number of users")

plt.show()

In [ ]:
Device_used = train_data.groupby('device_deviceCategory')['visitId'].nunique().sort_values(ascending=False).head(10)

Device_used.plot.bar()

plt.title("Device Used")
plt.xlabel("device category")
plt.ylabel("number of users")

plt.xticks(rotation=0)

plt.show()

Nonzero_revenue_device_used = train_data[train_data.totals_transactionRevenue>0].groupby('device_deviceCategory')['visitId'].nunique().sort_values(ascending=False).head(10)

Nonzero_revenue_device_used.plot.bar()

plt.title("Non Zero Revenue Device Used")
plt.xlabel("device category")
plt.ylabel("number of users")

plt.xticks(rotation=0)

plt.show()

In [ ]:
Top10_OS_used = train_data.groupby('device_operatingSystem')['visitId'].nunique().sort_values(ascending=False).head(10)

Top10_OS_used.plot.bar()

plt.title("Top 10 OS Used")
plt.xlabel("Device OS")
plt.ylabel("number of users")

plt.show()

Top10_nonzero_revenue_OS_used = train_data[train_data.totals_transactionRevenue>0].groupby('device_operatingSystem')['visitId'].nunique().sort_values(ascending=False).head(10)


new_index = ['Windows', 'Macintosh',  'Android', 'iOS', 'Linux', 'Chrome OS', 'Windows Phone']

Top10_nonzero_revenue_OS_used = Top10_nonzero_revenue_OS_used.reindex(new_index)

Top10_nonzero_revenue_OS_used.plot.bar()

plt.title("Top 7 Non Zero Revenue OS Used")
plt.xlabel("Device OS")
plt.ylabel("number of users")

plt.show()

In [ ]:
import matplotlib.pyplot as plt

Date_against_visitcount = train_data.groupby(['Date_Year', 'Date_Month', 'Date_Day'])['visitId'].nunique()

Date_against_visitcount.plot()

plt.title("Date against Visit Count")
plt.xlabel("Month")
plt.ylabel("Count")


plt.show()

In [ ]:
continent = train_data.groupby('geoNetwork_continent')['visitId'].nunique().sort_values(ascending=False).head(5)

continent.plot.bar()

plt.title("Access from different Continents")
plt.xlabel("Continent")
plt.ylabel("number of access")

plt.show()

continent_nonzero_revenue = train_data[train_data.totals_transactionRevenue>0].groupby('geoNetwork_continent')['visitId'].nunique().sort_values(ascending=False).head(5)

continent_nonzero_revenue.plot.bar()

plt.title("Non zero revenue access from different Continents")
plt.xlabel("Continent")
plt.ylabel("number of access")

plt.show()

In [ ]:
country = train_data.groupby('geoNetwork_country')['visitId'].nunique().sort_values(ascending=False).head(40)

country.plot.bar()

plt.title("Access from different Countries")
plt.xlabel("Country")
plt.ylabel("number of access")

plt.show()

country_nonzero_revenue = train_data[train_data.totals_transactionRevenue>0].groupby('geoNetwork_country')['visitId'].nunique().sort_values(ascending=False).head(10)

country_nonzero_revenue.plot.bar()

plt.title("Non zero revenue access from different Countries")
plt.xlabel("Country")
plt.ylabel("number of access")

plt.show()

In [ ]:
Top8_channel = train_data.groupby('channelGrouping')['visitId'].nunique().sort_values(ascending=False).head(10)
Top8_channel.plot.bar()

plt.title("Top 8 channel")
plt.xlabel("Channels")
plt.ylabel("number of accesses from channel")

plt.show()

Top8_channel_nonzero_revenue = train_data[train_data.totals_transactionRevenue>0].groupby('channelGrouping')['visitId'].nunique().sort_values(ascending=False).head(10)
Top8_channel_nonzero_revenue.plot.bar()

plt.title("Top 8 non zero revenue channel")
plt.xlabel("Channels")
plt.ylabel("number of accesses from channel")

plt.show()

In [ ]:
Top20_source = train_data.groupby('trafficSource_source')['visitId'].nunique().sort_values(ascending=False).head(20)
Top20_source.plot.bar()

plt.title("Top 20 source")
plt.xlabel("source")
plt.ylabel("number of access from source")


plt.show()

Top10_source_nonzero_revenue = train_data[train_data.totals_transactionRevenue>0].groupby('trafficSource_source')['visitId'].nunique().sort_values(ascending=False).head(10)
Top10_source_nonzero_revenue.plot.bar()

plt.title("Top 10 non zero reveue source")
plt.xlabel("source")
plt.ylabel("number of access from source")


plt.show()

In [ ]:
total_pageview = train_data.groupby('totals_pageviews')['visitId'].nunique().sort_values(ascending=False).head(60)
total_pageview.plot.bar()

plt.title("total count for page view")
plt.xlabel("number of page view")
plt.ylabel("total count")

plt.xticks(range(0,61, 10), range(0,61, 10), rotation=0) # ticks = ..., labels = ...


plt.show()

total_pageview_nonzero_revenue = train_data[train_data.totals_transactionRevenue>0].groupby('totals_pageviews')['visitId'].nunique().head(60)
total_pageview_nonzero_revenue.plot.bar()

plt.title("total count for non zero revenue page view")
plt.xlabel("number of page view")
plt.ylabel("total count")

plt.xticks(range(0,61, 10), range(0,61, 10), rotation=0) # ticks = ..., labels = ...

plt.show()

In [11]:
X_train= train_data.drop(columns=['totals_totalTransactionRevenue', 'totals_transactionRevenue']).apply(lambda x: pd.factorize(x)[0])
y_train = train_data['totals_transactionRevenue'].apply(lambda x: np.log1p(x))
X_test = test_data.drop(columns=['totals_totalTransactionRevenue', 'totals_transactionRevenue']).apply(lambda x: pd.factorize(x)[0])
y_test = test_data['totals_transactionRevenue'].apply(lambda x: np.log1p(x))


print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1708337, 52) (1708337,)
(401589, 52) (401589,)


In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50, max_depth=15,random_state=0)
clf.fit(X_train.values, y_train.values.astype('int'))
y_pred = clf.predict(X_test)
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

In [12]:
import xgboost as xgb

params = {'objective': 'reg:linear',
              'eval_metric': 'rmse',
              'eta': 0.001,
              'max_depth': 6,
              'subsample': 0.6,
              'colsample_bytree': 0.6,
              'alpha':0.001,
              'random_state': 42,
              'silent': True}


#gbm = xgb.XGBClassifier(max_depth=6, n_estimators=300, learning_rate=0.05)\
  #  .fit(X_train.values, y_train.values.astype('int'))
#y_pred = gbm.predict(test_X)

xgb_train_data = xgb.DMatrix(X_train[:1366669], y_train[:1366669])
xgb_val_data = xgb.DMatrix(X_train[1366669:], y_train[1366669:])
xgb_submit_data = xgb.DMatrix(X_test)

model = xgb.train(params, xgb_train_data, 
                      num_boost_round=200, 
                      evals= [(xgb_train_data, 'train'), (xgb_val_data, 'valid')],
                      early_stopping_rounds=10, 
                      verbose_eval=50
                     )

y_pred = model.predict(xgb_submit_data, ntree_limit=model.best_ntree_limit)

print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

[0]	train-rmse:1.87017	valid-rmse:1.85752
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.80364	valid-rmse:1.79146
[100]	train-rmse:1.74621	valid-rmse:1.73449
[150]	train-rmse:1.68648	valid-rmse:1.67521
[199]	train-rmse:1.62946	valid-rmse:1.61864
The rmse of prediction is: 1.7159866043568184
